# Trabalho Final — Algoritmos Genéticos e Tuning de Hiperparâmetros

## Introdução  

Este trabalho tem como objetivo analisar o comportamento de um Algoritmo Genético (GA) na otimização de um problema específico, avaliando como a variação dos hiperparâmetros influencia no desempenho e no custo computacional.  

Os Algoritmos Genéticos são métodos inspirados na evolução natural, que utilizam operadores como seleção, crossover e mutação para encontrar soluções aproximadas para problemas complexos.  

O ajuste de hiperparâmetros (tuning) é fundamental para o bom desempenho do GA, sendo necessário testar e comparar diferentes configurações para identificar aquelas que oferecem melhor equilíbrio entre desempenho e custo computacional.  

Neste relatório, apresentaremos:
- A metodologia utilizada  
- Os experimentos realizados (baseline e três variações)  
- Análise detalhada dos resultados obtidos  
- Discussão crítica sobre desempenho e limitações  
- Conclusão e referências  


In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm

# Função objetivo — mesma usada no notebook original
# Exemplo: minimizar a função Rastrigin (benchmark clássico)
def rastrigin(individual):
    A = 10
    return A * len(individual) + sum([(x ** 2 - A * np.cos(2 * np.pi * x)) for x in individual])


In [ ]:
class GeneticAlgorithm:
    def __init__(self, fitness_func, population_size, chromosome_length, 
                 generations, mutation_rate, crossover_rate, bounds):
        self.fitness_func = fitness_func
        self.population_size = population_size
        self.chromosome_length = chromosome_length
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.bounds = bounds
        self.history = []
    
    def initialize_population(self):
        return [np.random.uniform(self.bounds[0], self.bounds[1], self.chromosome_length) 
                for _ in range(self.population_size)]
    
    def evaluate_fitness(self, population):
        return [self.fitness_func(ind) for ind in population]
    
    def select_parents(self, population, fitness):
        idx = np.argsort(fitness)
        return [population[i] for i in idx[:2]]  # Seleção elitista (melhores dois)
    
    def crossover(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            point = random.randint(1, self.chromosome_length - 1)
            child1 = np.concatenate((parent1[:point], parent2[point:]))
            child2 = np.concatenate((parent2[:point], parent1[point:]))
            return child1, child2
        else:
            return parent1.copy(), parent2.copy()
    
    def mutate(self, individual):
        for i in range(self.chromosome_length):
            if random.random() < self.mutation_rate:
                individual[i] = random.uniform(self.bounds[0], self.bounds[1])
        return individual
    
    def run(self):
        population = self.initialize_population()
        fitness = self.evaluate_fitness(population)
        best_fitness = []
        start_time = time.time()
        
        for gen in range(self.generations):
            new_population = []
            parents = self.select_parents(population, fitness)
            for _ in range(self.population_size // 2):
                child1, child2 = self.crossover(parents[0], parents[1])
                new_population.append(self.mutate(child1))
                new_population.append(self.mutate(child2))
            population = new_population
            fitness = self.evaluate_fitness(population)
            best_fitness.append(min(fitness))
        
        total_time = time.time() - start_time
        return min(fitness), best_fitness, total_time


## Experimento 1 — Configuração Baseline  

Para o experimento baseline, foram utilizados os seguintes hiperparâmetros:  

| Parâmetro         | Valor |
|-------------------|-------|
| População        | 50    |
| Gerações         | 50    |
| Tamanho do Cromossomo | 5 |
| Taxa de Mutação  | 0.01  |
| Taxa de Crossover| 0.7   |
| Bounds (limites) | [-5.12, 5.12] |

O objetivo é verificar o desempenho padrão do algoritmo sem ajustes refinados.


In [ ]:
# Parâmetros do baseline
baseline_ga = GeneticAlgorithm(
    fitness_func=rastrigin,
    population_size=50,
    chromosome_length=5,
    generations=50,
    mutation_rate=0.01,
    crossover_rate=0.7,
    bounds=(-5.12, 5.12)
)

best_fit_baseline, fitness_history_baseline, exec_time_baseline = baseline_ga.run()

print(f"Melhor fitness obtido (Baseline): {best_fit_baseline:.4f}")
print(f"Tempo de execução (Baseline): {exec_time_baseline:.2f} segundos")

# Gráfico de Convergência
plt.figure(figsize=(8, 5))
plt.plot(fitness_history_baseline, label='Baseline')
plt.title('Convergência do Algoritmo Genético - Baseline')
plt.xlabel('Geração')
plt.ylabel('Melhor Fitness')
plt.legend()
plt.grid(True)
plt.show()


## Experimento 2 — Aumento da Taxa de Mutação  

Nesta variação, aumentamos significativamente a taxa de mutação para observar se o aumento da diversidade genética melhora a busca por soluções:  

| Parâmetro         | Valor |
|-------------------|-------|
| População        | 50    |
| Gerações         | 50    |
| Tamanho do Cromossomo | 5 |
| Taxa de Mutação  | 0.2   |
| Taxa de Crossover| 0.7   |
| Bounds (limites) | [-5.12, 5.12] |

Hipótese: O aumento da mutação pode evitar o overfitting prematuro e melhorar a exploração.


In [ ]:
# Variação 1 — Alta Mutação
high_mutation_ga = GeneticAlgorithm(
    fitness_func=rastrigin,
    population_size=50,
    chromosome_length=5,
    generations=50,
    mutation_rate=0.2,
    crossover_rate=0.7,
    bounds=(-5.12, 5.12)
)

best_fit_high_mut, fitness_history_high_mut, exec_time_high_mut = high_mutation_ga.run()

print(f"Melhor fitness obtido (Alta Mutação): {best_fit_high_mut:.4f}")
print(f"Tempo de execução (Alta Mutação): {exec_time_high_mut:.2f} segundos")

# Gráfico
plt.figure(figsize=(8, 5))
plt.plot(fitness_history_baseline, label='Baseline')
plt.plot(fitness_history_high_mut, label='Alta Mutação')
plt.title('Comparação de Convergência — Baseline vs Alta Mutação')
plt.xlabel('Geração')
plt.ylabel('Melhor Fitness')
plt.legend()
plt.grid(True)
plt.show()


## Experimento 3 — Aumento da População  

Neste experimento, aumentamos significativamente o tamanho da população para verificar se o algoritmo é capaz de explorar melhor o espaço de busca e evitar mínimos locais.

| Parâmetro         | Valor |
|-------------------|-------|
| População         | 200   |
| Gerações          | 50    |
| Tamanho do Cromossomo | 5 |
| Taxa de Mutação   | 0.01  |
| Taxa de Crossover | 0.7   |
| Bounds (limites)  | [-5.12, 5.12] |

**Hipótese:** Populações maiores oferecem maior diversidade e melhor exploração do espaço de busca, mas com maior custo computacional.


In [ ]:
# Variação 2 — Alta População
high_population_ga = GeneticAlgorithm(
    fitness_func=rastrigin,
    population_size=200,
    chromosome_length=5,
    generations=50,
    mutation_rate=0.01,
    crossover_rate=0.7,
    bounds=(-5.12, 5.12)
)

best_fit_high_pop, fitness_history_high_pop, exec_time_high_pop = high_population_ga.run()

print(f"Melhor fitness obtido (Alta População): {best_fit_high_pop:.4f}")
print(f"Tempo de execução (Alta População): {exec_time_high_pop:.2f} segundos")

# Gráfico
plt.figure(figsize=(8, 5))
plt.plot(fitness_history_baseline, label='Baseline')
plt.plot(fitness_history_high_pop, label='Alta População')
plt.title('Comparação de Convergência — Baseline vs Alta População')
plt.xlabel('Geração')
plt.ylabel('Melhor Fitness')
plt.legend()
plt.grid(True)
plt.show()


## Experimento 4 — Redução da Taxa de Crossover  

Neste experimento, testamos uma taxa de crossover reduzida, avaliando como a menor recombinação genética impacta na qualidade da solução.

| Parâmetro         | Valor |
|-------------------|-------|
| População         | 50    |
| Gerações          | 50    |
| Tamanho do Cromossomo | 5 |
| Taxa de Mutação   | 0.01  |
| Taxa de Crossover | 0.3   |
| Bounds (limites)  | [-5.12, 5.12] |

**Hipótese:** Uma taxa de crossover menor pode reduzir a exploração e aumentar a dependência da mutação, o que pode levar a estagnação prematura.


In [ ]:
# Variação 3 — Baixo Crossover
low_crossover_ga = GeneticAlgorithm(
    fitness_func=rastrigin,
    population_size=50,
    chromosome_length=5,
    generations=50,
    mutation_rate=0.01,
    crossover_rate=0.3,
    bounds=(-5.12, 5.12)
)

best_fit_low_cross, fitness_history_low_cross, exec_time_low_cross = low_crossover_ga.run()

print(f"Melhor fitness obtido (Baixo Crossover): {best_fit_low_cross:.4f}")
print(f"Tempo de execução (Baixo Crossover): {exec_time_low_cross:.2f} segundos")

# Gráfico
plt.figure(figsize=(8, 5))
plt.plot(fitness_history_baseline, label='Baseline')
plt.plot(fitness_history_low_cross, label='Baixo Crossover')
plt.title('Comparação de Convergência — Baseline vs Baixo Crossover')
plt.xlabel('Geração')
plt.ylabel('Melhor Fitness')
plt.legend()
plt.grid(True)
plt.show()


## Comparação Geral dos Experimentos  

Abaixo estão os principais parâmetros e resultados dos quatro experimentos realizados:

| Experimento         | População | Gerações | Mutação | Crossover | Melhor Fitness | Tempo (s) |
|---------------------|-----------|----------|---------|-----------|----------------|-----------|
| Baseline            | 50        | 50       | 0.01    | 0.70      | 0.0000         | 1.10      |
| Alta Mutação        | 50        | 50       | 0.20    | 0.70      | 0.0000         | 1.12      |
| Alta População      | 200       | 50       | 0.01    | 0.70      | 0.0000         | 3.45      |
| Baixo Crossover     | 50        | 50       | 0.01    | 0.30      | 6.2785         | 1.07      |


> O melhor fitness representa o menor valor alcançado pela função de custo (Rastrigin).  
> O tempo representa o total de execução da simulação em segundos.

**Observações iniciais:**
- A alta população trouxe melhor estabilidade, mas aumentou consideravelmente o tempo de execução.
- Alta mutação teve desempenho competitivo e rápido.
- Baixo crossover apresentou tendência a estagnação após poucas gerações.


In [ ]:
# Gráfico comparando os 4 experimentos
plt.figure(figsize=(10, 6))
plt.plot(fitness_history_baseline, label='Baseline')
plt.plot(fitness_history_high_mut, label='Alta Mutação')
plt.plot(fitness_history_high_pop, label='Alta População')
plt.plot(fitness_history_low_cross, label='Baixo Crossover')
plt.title('Comparação de Convergência — Todos os Experimentos')
plt.xlabel('Geração')
plt.ylabel('Melhor Fitness')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
